# Investigating `ecg_sample_v3.zip`

See `ecg_sample/dbo.ECG01.Table.sql`

In [11]:
import io
import glob
import zlib

from xml.dom.minidom import parseString

import tqdm
import joblib

In [2]:
with io.open("ecg_sample/dbo.ECG01.Table.sql", encoding="utf-16") as f:
    lines = f.readlines()
print("".join(lines[:28]))  # anything more is an error
inserts = [l for l in lines[28:] if l != "GO\n"]

DB_COLS = (
    "ecgId",
    "dateAcquired",
    "dateReceived",
    "dateConfirmed",
    "UserField4",
    "severityId",
    "headerInfo",
    "userDefines",
    "orderInfo",
    "documentInfo",
    "reportInfo",
    "acquisitionInfo",
    "interpretationInfo",
    "age",
    "ageUnits",
    "Sex",
    "waveformInfo",
    "measurementInfo"
)

USE [dbems_prodcopy]
GO
/****** Object:  Table [dbo].[ECG01]    Script Date: 2020-10-20 7:08:09 PM ******/
SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO
CREATE TABLE [dbo].[ECG01](
	[ecgId] [uniqueidentifier] NOT NULL,
	[dateAcquired] [datetime] NOT NULL,
	[dateReceived] [datetime] NOT NULL,
	[dateConfirmed] [datetime] NULL,
	[UserField4] [nvarchar](32) NULL,
	[severityId] [smallint] NULL,
	[headerInfo] [varbinary](4000) NOT NULL,
	[userDefines] [varbinary](4000) NULL,
	[orderInfo] [varbinary](4000) NULL,
	[documentInfo] [varbinary](4000) NOT NULL,
	[reportInfo] [varbinary](4000) NOT NULL,
	[acquisitionInfo] [varbinary](4000) NOT NULL,
	[interpretationInfo] [varbinary](4000) NULL,
	[age] [float] NULL,
	[ageUnits] [tinyint] NULL,
	[Sex] [tinyint] NULL,
	[waveformInfo] [image] NULL,
	[measurementInfo] [varbinary](8000) NULL
) ON [PRIMARY] TEXTIMAGE_ON [PRIMARY]
GO



In [3]:
def convert_line_to_payload(raw_line):
    line = raw_line.rstrip()
    raw_insert, raw_values = line.split(" VALUES ")
    raw_columns = raw_insert[len("INSERT [dbo].[ECG01] ("):len(raw_insert) - len(")")]  # raw column names
    raw_values = (raw_values[1:len(raw_values) - 1])
    
    raw_columns = raw_columns.split(", ")
    columns = tuple(col[1:len(col) - 1] for col in raw_columns)  # remove surrounding square braces `[`, `]`
    values = raw_values.split(", ")  # , raw_values.split(", "))

    assert columns == DB_COLS, "col mismatch"
    assert len(values) == len(columns), "not enough values for columns"

    return dict(zip(columns, values))


def convert_payload_to_xml(payload):
    xml = {}
    for key, value in payload.items():
        if value.startswith("0x"):
            if key == "waveformInfo":
                val_bytes = zlib.decompress(bytes.fromhex(value[2:]))
                xml[key] = val_bytes.decode("utf-8")
            else:
                val_bytes = zlib.decompress(bytes.fromhex(value[2:]))
                xml[key] = val_bytes.decode("utf-16")
    return xml


def convert_xml_fragments_to_string(xml_fragments):
    xml_string = "".join([
        xml_fragments["headerInfo"],
        xml_fragments["orderInfo"],
        xml_fragments["documentInfo"],
        xml_fragments["reportInfo"],
        xml_fragments["acquisitionInfo"],
        xml_fragments["interpretationInfo"],
        xml_fragments["userDefines"],
        xml_fragments["measurementInfo"],
        xml_fragments["waveformInfo"],
        "</restingecgdata>"
    ])
    return xml_string
    

In [4]:
def convert_and_save_xml(idx_raw_insert):
    ins_idx, raw_insert = idx_raw_insert
    try:
        payload = convert_line_to_payload(raw_insert)
        xml_fragments = convert_payload_to_xml(payload)
        xml_string = convert_xml_fragments_to_string(xml_fragments)
        xml_dom = parseString(xml_string)
        
        # save to file
        ecgId = payload["ecgId"][2:len(payload["ecgId"]) - 1]  # remove enclosing `N'`, `'`
        with open(f"ecg_sample/xml_output/{ecgId}.xml", "w") as f:
            f.write(xml_dom.toprettyxml())
        return None
    except Exception:
        return ins_idx


In [5]:
failed = joblib.Parallel(n_jobs=-1, verbose=10)(
    joblib.delayed(convert_and_save_xml)(idx_raw_insert) for idx_raw_insert in enumerate(inserts)
)
failed = [fail for fail in failed if fail != None]
failed

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 96 concurrent workers.
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    1.6s
[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed:    1.7s
[Parallel(n_jobs=-1)]: Done  50 tasks      | elapsed:    1.7s
[Parallel(n_jobs=-1)]: Done  73 tasks      | elapsed:    1.7s
[Parallel(n_jobs=-1)]: Done  96 tasks      | elapsed:    1.7s
[Parallel(n_jobs=-1)]: Done 121 tasks      | elapsed:    1.7s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done 173 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done 200 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.1960s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Done 229 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done 258 tasks      | elapsed:    1.9s
[Parallel(n_jobs=-1)]: Done 289 tasks      | elapsed:    1.9s
[Parallel(n_jobs=-1)]: Done 320 tasks      | elapsed:    1.9s
[Parallel(n_jobs=-1)]: Done 353 ta

[]

# Convert to PDF

In [16]:
import argparse
import os
import re
from functools import partial
from multiprocessing import Pool, freeze_support
from xml.etree import ElementTree as ET

from matplotlib import pyplot as plt
from PySierraECG import get_leads
from tqdm import tqdm


In [17]:
xml_files = glob.glob("ecg_sample/xml_output/*.xml")
print(len(xml_files))

2054


In [18]:
def get_tree_val(tree, tree_key, p_key="globalmeasurements", ns={"ns": "http://www3.medical.philips.com"}):
    c = tree.find(f".//ns:{p_key}/ns:{tree_key}", namespaces=ns)
    if c is None:
        return None
    else:
        return c.text


def calculate_tick_range(min_y, max_y, step):
    y_min_offset = min_y % step
    y_max_offset = max_y % step
    return range(int(min_y - y_min_offset), int(max_y - y_max_offset + step), step)


In [33]:
def generate_pdf(xml_fn, out_dir="ecg_sample/pdf_output", tick_aspect=10/4, main_lead_key="II"):
    # check if ecg_id filename is valid
    r = re.match(
        r".*(?P<id>[\w]{8}-[\w]{4}-[\w]{4}-[\w]{4}-[\w]{12})\.xml", xml_fn)
    if not r:
        return None, None
    ecg_id = r.group("id")
    
    warn = None

    # check if we can get the leads from the filename
    try:
        leads = get_leads(xml_fn)
    except Exception as e:
        return ecg_id, str(e)

    # check if we can load and parse the XML
    try:
        tree = ET.parse(xml_fn)
    except Exception as e:
        return ecg_id, str(e)

    # check durations and number of samples match the data array
    try:
        assert len(leads) == 12, "lead count not equal to 12"
        ref_ms_per_sample = None
        names = ["I", "II", "III", "aVR", "aVL",
                 "aVF", "V1", "V2", "V3", "V4", "V5", "V6"]
        for lead in leads:
            ms_per_sample = lead["duration"] // lead["nsamples"]
            if ref_ms_per_sample is None:
                ref_ms_per_sample = ms_per_sample
            else:
                assert ms_per_sample == ref_ms_per_sample, "ms per sample mismatch"
            x = range(0, lead["duration"], ms_per_sample)
            assert len(x) == lead["nsamples"], "duration nsamples mismatch"
            assert lead["name"] in names, f"invalid lead name: {lead['name']}"
            # assert max(lead['data']) <= 1000, "WARN: lead signal max over 5mV"
            # assert min(lead['data']) >= -1000, "WARN: lead signal min under -5mV"
            names.pop(names.index(lead["name"]))
    except Exception as e:
        return ecg_id, str(e)
    finally:
        del names
        del ref_ms_per_sample

    leads_dict = {}
    while leads:
        lead = leads.pop()
        name = lead.pop("name")
        leads_dict[name] = lead["data"]
    del leads
    del name

    # according to the calibration signal, 0-200 is equal to 1mV
    # we want to make a grid where all of the ticks are in millimeters
    # standard calibration: 10mm/mv (y-axis), 25mm/sec (x-axis)
    # y: 5mm == 100 units; x: 5mm == 40 ms

    # I,   aVR, V1, V4
    # II,  aVL, V2, V5
    # III, aVF, V3, V6
    # II

    q_x = len(x) // 4   # plot column separator width
    q_y = 600           # plot row separator height

    plot_leads = {
        "MAIN": leads_dict[main_lead_key],
        "I":    [v + 3 * q_y for v in leads_dict["I"]][:q_x],
        "II":   [v + 2 * q_y for v in leads_dict["II"]][:q_x],
        "III":  [v + 1 * q_y for v in leads_dict["III"]][:q_x],
        "aVR":  [v + 3 * q_y for v in leads_dict["aVR"]][q_x:2 * q_x],
        "aVL":  [v + 2 * q_y for v in leads_dict["aVL"]][q_x:2 * q_x],
        "aVF":  [v + 1 * q_y for v in leads_dict["aVF"]][q_x:2 * q_x],
        "V1":   [v + 3 * q_y for v in leads_dict["V1"]][2 * q_x:3 * q_x],
        "V2":   [v + 2 * q_y for v in leads_dict["V2"]][2 * q_x:3 * q_x],
        "V3":   [v + 1 * q_y for v in leads_dict["V3"]][2 * q_x:3 * q_x],
        "V4":   [v + 3 * q_y for v in leads_dict["V4"]][3 * q_x:4 * q_x],
        "V5":   [v + 2 * q_y for v in leads_dict["V5"]][3 * q_x:4 * q_x],
        "V6":   [v + 1 * q_y for v in leads_dict["V6"]][3 * q_x:4 * q_x]
    }

    # some of the data has signals that are extreme outliers and overlap over the other leads
    y_min = min([min(l) for l in plot_leads.values()])
    y_max = max([max(l) for l in plot_leads.values()])
    x = range(0, lead["duration"], lead["duration"] // lead["nsamples"])

    # if the scaled ecg signal y-axis is longer than the time x-axis, that's an error (breaks all text)
    if (y_max - y_min) * (tick_aspect) > max(x):
        # return ecg_id, "mv signal has greater distribution than allowable ms time range"
        warn = "WARN: mv signal has greater distribution than allowable ms time range"

    fig, ax = plt.subplots(figsize=(10.5 * 2, 8.0 * 2), dpi=300)
    ax.plot(x, plot_leads["MAIN"])
    ax.set_xticks(range(0, max(x)+1, 40), minor=True)
    ax.set_xticks(range(0, max(x)+1, 200), minor=False)

    # plot columns [I, II, III]
    ax.plot(x[:q_x], plot_leads["I"])
    ax.plot(x[:q_x], plot_leads["II"])
    ax.plot(x[:q_x], plot_leads["III"])
    # plot columns [aVR, aVL, aVF]
    ax.plot(x[q_x:2 * q_x], plot_leads["aVR"])
    ax.plot(x[q_x:2 * q_x], plot_leads["aVL"])
    ax.plot(x[q_x:2 * q_x], plot_leads["aVF"])
    # plot columns [V1, V2, V3]
    ax.plot(x[2 * q_x:3 * q_x], plot_leads["V1"])
    ax.plot(x[2 * q_x:3 * q_x], plot_leads["V2"])
    ax.plot(x[2 * q_x:3 * q_x], plot_leads["V3"])
    # plot columns [V4, V5, V6]
    ax.plot(x[3 * q_x:4 * q_x], plot_leads["V4"])
    ax.plot(x[3 * q_x:4 * q_x], plot_leads["V5"])
    ax.plot(x[3 * q_x:4 * q_x], plot_leads["V6"])

    # set label text for each lead
    q_xd = max(x) // 4
    text_kwargs = {"horizontalalignment": "left",
                   "verticalalignment": "top", "fontsize": 14}
    ax.text(0 * q_xd, 0 * q_y + 200, main_lead_key, **text_kwargs)  # MAIN
    ax.text(0 * q_xd, 3 * q_y + 200, "I", **text_kwargs)
    ax.text(0 * q_xd, 2 * q_y + 200, "II", **text_kwargs)
    ax.text(0 * q_xd, 1 * q_y + 200, "III", **text_kwargs)
    ax.text(1 * q_xd, 3 * q_y + 200, "aVR", **text_kwargs)
    ax.text(1 * q_xd, 2 * q_y + 200, "aVL", **text_kwargs)
    ax.text(1 * q_xd, 1 * q_y + 200, "aVF", **text_kwargs)
    ax.text(2 * q_xd, 3 * q_y + 200, "V1", **text_kwargs)
    ax.text(2 * q_xd, 2 * q_y + 200, "V2", **text_kwargs)
    ax.text(2 * q_xd, 1 * q_y + 200, "V3", **text_kwargs)
    ax.text(3 * q_xd, 3 * q_y + 200, "V4", **text_kwargs)
    ax.text(3 * q_xd, 2 * q_y + 200, "V5", **text_kwargs)
    ax.text(3 * q_xd, 1 * q_y + 200, "V6", **text_kwargs)

    # major y-ticks every 100, minor y-ticks every 20. major ticks aligned to 0
    # ax.set_yticks(range(-100, 250+1, 20), minor=True)
    # ax.set_yticks(range(-100, 250+1, 100), minor=False)
    ax.set_yticks(calculate_tick_range(y_min, y_max, 20), minor=True)
    ax.set_yticks(calculate_tick_range(y_min, y_max, 100), minor=False)
    ax.set_xticklabels(())
    ax.set_yticklabels(())

    # set the gridlines
    ax.grid(b=True, which="major", axis="both",
            color="pink", linestyle="solid", linewidth=1.5)
    ax.grid(b=True, which="minor", axis="both",
            color="pink", linestyle="solid", linewidth=0.5)

    # force aspect ratio to have gridlines square box shaped
    ax.set_aspect(tick_aspect)
    plt.margins(0.01)

    ax.set_xlabel("25 ticks/second")
    ax.set_ylabel("10 ticks/mV")

    # hardcoded text metadata, assumes 11000 millisecond duration for x-axis offset
    x_off = 0
    text_kwargs = {"horizontalalignment": "left",
                   "verticalalignment": "bottom", "fontsize": 10}
    ns = {"ns": "http://www3.medical.philips.com"}

    # construct acquisition & patient information
    ecg_metadata = f"ecgId: {ecg_id}\n"
    ecg_metadata += "\nAcquisition"
    ecg_metadata += "\n  Date:"
    ecg_metadata += "\n  Time:"
    ecg_metadata += "\nPatient ID:"
    ecg_metadata += "\nBirth Date:"
    ecg_metadata += "\nSex:"
    ecg_metadata += "\nHeight:"
    ecg_metadata += "\nWeight:"
    ax.text(x_off, y_max + 50, ecg_metadata, **text_kwargs)
    x_off += 650

    patientid = get_tree_val(tree, "patientid", p_key="generalpatientdata")
    acq_node = tree.find("./ns:dataacquisition", namespaces=ns)
    acq_date = acq_node.attrib.get("date", None)
    acq_time = acq_node.attrib.get("time", None)
    ecg_metadata = f"\n{acq_date}\n{acq_time}"
    ecg_metadata += f"\n{patientid}"
    dateofbirth = get_tree_val(tree, "dateofbirth", p_key="age")
    ecg_metadata += f"\n{dateofbirth}"
    sex = get_tree_val(tree, "sex", p_key="generalpatientdata")
    ecg_metadata += f"\n{sex}"
    height = get_tree_val(tree, "cm", p_key="height")
    ecg_metadata += f"\n{height} cm"
    weight = get_tree_val(tree, "kg", p_key="weight")
    ecg_metadata += f"\n{weight} kg"
    ax.text(x_off, y_max + 50, ecg_metadata, **text_kwargs)
    x_off += 850

    # construct duration
    ecg_metadata = "\nRR Interval:"
    ecg_metadata += "\nP Duration:"
    ecg_metadata += "\nPR Interval:"
    ecg_metadata += "\nQRS Duration:"
    ecg_metadata += "\nQT Interval:"
    ecg_metadata += "\nQTCb:"
    ecg_metadata += "\nQTCf:"
    ecg_metadata += "\nQ Onset:"
    ax.text(x_off, y_max + 50, ecg_metadata, **text_kwargs)
    x_off += 750
    rrint = get_tree_val(tree, "rrint")
    ecg_metadata = f"\n{rrint} ms"
    pdur = get_tree_val(tree, "pdur")
    ecg_metadata += f"\n{pdur} ms"
    pr_int = get_tree_val(tree, "print")
    ecg_metadata += f"\n{pr_int} ms"
    qrs_dur = get_tree_val(tree, "qrsdur")
    ecg_metadata += f"\n{qrs_dur} ms"
    qt_int = get_tree_val(tree, "qtint")
    ecg_metadata += f"\n{qt_int} ms"
    qtcb = get_tree_val(tree, "qtcb")
    ecg_metadata += f"\n{qtcb} ms"
    qtcf = get_tree_val(tree, "qtcf")
    ecg_metadata += f"\n{qtcf} ms"
    qonset = get_tree_val(tree, "qonset")
    ecg_metadata += f"\n{qonset} ms"
    ax.text(x_off, y_max + 50, ecg_metadata, **text_kwargs)
    x_off += 800

    # construct frontaxis
    ecg_metadata = "Heart rate:\n"
    ecg_metadata += "\nP Front Axis:"
    ecg_metadata += "\ni40 Front Axis:"
    ecg_metadata += "\nt40 Front Axis:"
    ecg_metadata += "\nQRS Front Axis:"
    ecg_metadata += "\nST Front Axis:"
    ecg_metadata += "\nT Front Axis:"
    ax.text(x_off, y_max + 50, ecg_metadata, **text_kwargs)
    x_off += 800
    heartrate = get_tree_val(tree, "heartrate")
    ecg_metadata = f"{heartrate} BPM\n"
    pfrontaxis = get_tree_val(tree, "pfrontaxis")
    ecg_metadata += f"\n{pfrontaxis}°"
    i40frontaxis = get_tree_val(tree, "i40frontaxis")
    ecg_metadata += f"\n{i40frontaxis}°"
    t40frontaxis = get_tree_val(tree, "t40frontaxis")
    ecg_metadata += f"\n{i40frontaxis}°"
    qrsfrontaxis = get_tree_val(tree, "qrsfrontaxis")
    ecg_metadata += f"\n{qrsfrontaxis}°"
    stfrontaxis = get_tree_val(tree, "stfrontaxis")
    ecg_metadata += f"\n{stfrontaxis}°"
    tfrontaxis = get_tree_val(tree, "tfrontaxis")
    ecg_metadata += f"\n{i40frontaxis}°"
    ax.text(x_off, y_max + 50, ecg_metadata, **text_kwargs)
    x_off += 800

    # construct horizaxis
    ecg_metadata = "Atrial rate:\n"
    ecg_metadata += "\nP Horiz. Axis:"
    ecg_metadata += "\ni40 Horiz. Axis:"
    ecg_metadata += "\nt40 Horiz. Axis:"
    ecg_metadata += "\nQRS Horiz. Axis:"
    ecg_metadata += "\nST Horiz. Axis:"
    ecg_metadata += "\nT Horiz. Axis:"
    ax.text(x_off, y_max + 50, ecg_metadata, **text_kwargs)
    x_off += 800
    atrialrate = get_tree_val(tree, "atrialrate")
    ecg_metadata = f"{atrialrate} BPM\n"
    phorizaxis = get_tree_val(tree, "phorizaxis")
    ecg_metadata += f"\n{phorizaxis}°"
    i40horizaxis = get_tree_val(tree, "i40horizaxis")
    ecg_metadata += f"\n{i40horizaxis}°"
    t40horizaxis = get_tree_val(tree, "t40horizaxis")
    ecg_metadata += f"\n{i40horizaxis}°"
    qrshorizaxis = get_tree_val(tree, "qrshorizaxis")
    ecg_metadata += f"\n{qrshorizaxis}°"
    sthorizaxis = get_tree_val(tree, "sthorizaxis")
    ecg_metadata += f"\n{sthorizaxis}°"
    thorizaxis = get_tree_val(tree, "thorizaxis")
    ecg_metadata += f"\n{i40horizaxis}°"
    ax.text(x_off, y_max + 50, ecg_metadata, **text_kwargs)
    x_off += 800

    # construct severity and codes
    severity = get_tree_val(tree, "severity", p_key="interpretation")
    ecg_metadata = f"{severity}\n"
    mdsignatureline = get_tree_val(tree, "mdsignatureline", p_key="interpretation")
    ecg_metadata += f"{mdsignatureline}\n"
    codes = zip(
        [x.text or "" for x in tree.findall(
            f".//ns:statement/ns:statementcode", namespaces=ns)],
        [x.text or "" for x in tree.findall(
            f".//ns:statement/ns:leftstatement", namespaces=ns)],
        [x.text or "" for x in tree.findall(
            f".//ns:statement/ns:rightstatement", namespaces=ns)]
    )
    for code in codes:
        ecg_metadata += "\n" + " ".join(code)
    ax.text(x_off, y_max + 50, ecg_metadata, **text_kwargs)

    # plt.show()
    fig_name = os.path.join(out_dir, f"{ecg_id}.pdf")
    fig.savefig(fig_name, bbox_inches='tight', pad_inches=0.1)
    plt.close(fig)
    return ecg_id, warn


In [34]:
pdf_gen_output = joblib.Parallel(n_jobs=-1, verbose=10)(
    joblib.delayed(generate_pdf)(xml_fp) for xml_fp in xml_files
)
pdf_gen_output = dict(pdf_gen_output)
# pdf_gen_output
# failed = dict(()
# failed
warn_fail = dict((k, v) for (k, v) in pdf_gen_output.items() if v != None)
warn_fail

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 96 concurrent workers.
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    4.5s
[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed:    4.6s
[Parallel(n_jobs=-1)]: Done  50 tasks      | elapsed:    4.7s
[Parallel(n_jobs=-1)]: Done  73 tasks      | elapsed:    4.8s
[Parallel(n_jobs=-1)]: Done  96 tasks      | elapsed:    5.1s
[Parallel(n_jobs=-1)]: Done 121 tasks      | elapsed:    7.2s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:    7.3s
[Parallel(n_jobs=-1)]: Done 173 tasks      | elapsed:    7.5s
[Parallel(n_jobs=-1)]: Done 200 tasks      | elapsed:    9.5s
[Parallel(n_jobs=-1)]: Done 229 tasks      | elapsed:    9.8s
[Parallel(n_jobs=-1)]: Done 258 tasks      | elapsed:   10.0s
[Parallel(n_jobs=-1)]: Done 289 tasks      | elapsed:   11.2s
[Parallel(n_jobs=-1)]: Done 320 tasks      | elapsed:   12.3s
[Parallel(n_jobs=-1)]: Done 353 tasks      | elapsed:   12.6s
[Parallel(n_jobs=-1)]: Done 386 tasks      | elapsed:  

{'c766c180-f408-11e7-4823-0005039b0029': 'WARN: mv signal has greater distribution than allowable ms time range',
 '52737a80-f296-11e7-4823-001943f70029': 'WARN: mv signal has greater distribution than allowable ms time range',
 '9172e300-f238-11e7-4823-0005ee5d0029': 'WARN: mv signal has greater distribution than allowable ms time range',
 'b440fb00-f3e2-11e7-4823-0004c2b00029': 'WARN: mv signal has greater distribution than allowable ms time range'}